# Working with CDP Transcripts

Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

# Connecting to resources

Having access to both the CDP instance's database and file store with make accessing and using the transcripts easiest. It is recommended to read the database and file store usage notebooks prior to working through this one.

For details on database usage, refer to the notebook example on database basics [here](./database.ipynb).

For details on file store usage, refer to the notebook example on file store basics [here](./file_store.ipynb).

**Note:** This notebook connects to the staging instance of Seattle's Firestore database and file store. To use production data, connect to the Cloud Firestore instance: `cdp-seattle`. To use production files, connect to the GCS instance: `cdp-seattle.appspot.com`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase
from cdptools.file_stores.gcs_file_store import GCSFileStore
import pandas as pd

db = CloudFirestoreDatabase("stg-cdp-seattle")
fs = GCSFileStore("stg-cdp-seattle.appspot.com")
db, fs

(<CloudFirestoreDatabase [stg-cdp-seattle]>,
 <GCSFileStore [stg-cdp-seattle.appspot.com]>)

### Find all transcripts

Simple query the transcript table!

In [2]:
transcripts = pd.DataFrame(db.select_rows_as_list("transcript"))
transcripts.head()

,confidence,created,event_id,file_id,transcript_id
0,0.935917,2019-06-04 00:49:28.535684,5970315e-cc33-4d84-8ee0-131274f2076d,c892939c-d84e-495c-a3ce-75526f3d7237,001909ae-f491-4fc4-8188-85aa4d524c83
1,0.924720,2019-06-04 00:04:34.542851,f39c25f4-1d9e-40f4-97b1-910189bc1fe8,7d6d7bb8-ba51-4883-abbe-cbbdf54989fc,19e02e0e-b634-4f2c-9ec7-5d2abdfbbb6f
2,0.941460,2019-06-04 00:36:34.015228,d4dd567d-3f67-4e18-8249-3eaf115a5a6f,5c2cbe25-347c-49c6-88ad-caaac309125c,3d87fb86-a7a5-4980-af64-69be8915d4e9
3,0.936662,2019-06-04 01:05:24.504101,83c21b25-47f4-4ef4-988e-49eae732e8d8,e3b56d4a-b66b-46dd-9a84-df2c2106a7c1,4f6d7c24-e7d0-4e28-a2b5-b3f97f19bda5
4,0.944922,2019-06-04 01:00:48.302970,0855398b-ea28-4053-a866-f40c09d11d94,732fad49-9c1a-4307-9cd4-b59bbba8716a,5cf33dff-4527-41b1-8d05-bbc0fcf85904


### Join file, event, and body information

While the above results are somewhat useful, we should probably merge information from the other tables too...

In [3]:
# Get the other tables
events = pd.DataFrame(db.select_rows_as_list("event"))
bodies = pd.DataFrame(db.select_rows_as_list("body"))
files = pd.DataFrame(db.select_rows_as_list("file"))

# Merge the tables
events = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
transcripts = transcripts.merge(files, left_on="file_id", right_on="file_id", suffixes=("_transcript", "_file"))
transcripts = transcripts.merge(events, left_on="event_id", right_on="event_id", suffixes=("_transcript", "_event"))
transcripts.head()

,confidence,created_transcript,event_id,file_id,transcript_id,content_type,created_file,description_transcript,filename,uri,...,created_event,event_datetime,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description_event,name
0,0.935917,2019-06-04 00:49:28.535684,5970315e-cc33-4d84-8ee0-131274f2076d,c892939c-d84e-495c-a3ce-75526f3d7237,001909ae-f491-4fc4-8188-85aa4d524c83,None,2019-06-04 00:47:41.587430,None,15a707389674d13917b416d5cb51122f5824ec2450b90a...,gs://stg-cdp-seattle.appspot.com/15a707389674d...,...,2019-06-04 00:47:47.028966,2019-05-28 14:00:00,3964,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...,2019-06-04 00:03:50.029975,None,City Council
1,0.924720,2019-06-04 00:04:34.542851,f39c25f4-1d9e-40f4-97b1-910189bc1fe8,7d6d7bb8-ba51-4883-abbe-cbbdf54989fc,19e02e0e-b634-4f2c-9ec7-5d2abdfbbb6f,None,2019-06-04 00:03:46.726894,None,5912ab2bdb38aedadbf98de09f1fc5dd274b0e317ea7a3...,gs://stg-cdp-seattle.appspot.com/5912ab2bdb38a...,...,2019-06-04 00:03:50.288189,2019-06-03 14:00:00,3975,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...,2019-06-04 00:03:50.029975,None,City Council
2,0.941460,2019-06-04 00:36:34.015228,d4dd567d-3f67-4e18-8249-3eaf115a5a6f,5c2cbe25-347c-49c6-88ad-caaac309125c,3d87fb86-a7a5-4980-af64-69be8915d4e9,None,2019-06-04 00:36:08.966368,None,7cf67bd8a47ecc57c8e66efefa2153bdc48496346b07aa...,gs://stg-cdp-seattle.appspot.com/7cf67bd8a47ec...,...,2019-06-04 00:36:14.180885,2019-05-22 12:30:00,3958,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-06-04 00:36:13.194924,None,"Civil Rights, Utilities, Economic Development,..."
3,0.936662,2019-06-04 01:05:24.504101,83c21b25-47f4-4ef4-988e-49eae732e8d8,e3b56d4a-b66b-46dd-9a84-df2c2106a7c1,4f6d7c24-e7d0-4e28-a2b5-b3f97f19bda5,None,2019-06-04 01:04:47.738380,None,9d9fa6aa5aca27f76e3efe1685738f6e5df6d40a2ac674...,gs://stg-cdp-seattle.appspot.com/9d9fa6aa5aca2...,...,2019-06-04 01:05:12.913600,2019-05-28 09:30:00,3966,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/su...,2019-06-04 00:40:31.798002,None,Council Briefing
4,0.944922,2019-06-04 01:00:48.302970,0855398b-ea28-4053-a866-f40c09d11d94,732fad49-9c1a-4307-9cd4-b59bbba8716a,5cf33dff-4527-41b1-8d05-bbc0fcf85904,None,2019-06-04 01:00:27.258083,None,12d0531ebcb93e818d78da75a3171184d8e290a45948f6...,gs://stg-cdp-seattle.appspot.com/12d0531ebcb93...,...,2019-06-04 01:00:32.901831,2019-06-03 09:30:00,3972,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/CouncilBriefings...,http://video.seattle.gov:8080/media/council/br...,2019-06-04 00:40:31.798002,None,Council Briefing


### Downloading and reading a transcript

A manifest is great, but you probably want to work with the transcripts locally.

In [4]:
# Download to local
save_path = fs.download_file(transcripts.loc[3]["filename"])
save_path

PosixPath('/Users/maxfield/Desktop/active/cdp/cdptools/examples/9d9fa6aa5aca27f76e3efe1685738f6e5df6d40a2ac6741fe353f014fce9f56d_ts_sentences_transcript_0.txt')

In [5]:
# Read the transcript
import json
with open(save_path, "r") as read_in:
    sentences = json.load(read_in)
    for s in sentences[:3]:
        print(s)

{'sentence': 'Good afternoon, everybody Welcome to the sustainability of Transportation committee.', 'start_time': 41.5, 'end_time': 46.2}
{'sentence': "And it's Wednesday May 29th 2019.", 'start_time': 46.2, 'end_time': 48.7}
{'sentence': "It's 2:03 p.m.", 'start_time': 48.7, 'end_time': 50.0}


### Handling multiple transcripts for a single event

A single event may have multiple versions of a transcripts. In this case you can decide how you want to filter them down, but generally, a decent solution is to group by `id_event`  then choose the most recent. You can do this with `pandas.Series.idxmax()`.

[stackoverflow](https://stackoverflow.com/questions/10202570/pandas-dataframe-find-row-where-values-for-column-is-maximal)

In [6]:
# Group rows
most_recent_transcripts = []
grouped = transcripts.groupby("event_id")
for name, group in grouped:
    most_recent = group.loc[group["created_transcript"].idxmax()]
    most_recent_transcripts.append(most_recent)
    
most_recent = pd.DataFrame(most_recent_transcripts)
most_recent.head()

,confidence,created_transcript,event_id,file_id,transcript_id,content_type,created_file,description_transcript,filename,uri,...,created_event,event_datetime,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description_event,name
4,0.944922,2019-06-04 01:00:48.302970,0855398b-ea28-4053-a866-f40c09d11d94,732fad49-9c1a-4307-9cd4-b59bbba8716a,5cf33dff-4527-41b1-8d05-bbc0fcf85904,None,2019-06-04 01:00:27.258083,None,12d0531ebcb93e818d78da75a3171184d8e290a45948f6...,gs://stg-cdp-seattle.appspot.com/12d0531ebcb93...,...,2019-06-04 01:00:32.901831,2019-06-03 09:30:00,3972,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/CouncilBriefings...,http://video.seattle.gov:8080/media/council/br...,2019-06-04 00:40:31.798002,None,Council Briefing
5,0.955914,2019-06-04 00:40:48.990044,47fb22e5-93f8-4e2f-a707-1152fe671a0c,337cabb1-e040-4bd3-ad03-184065148593,638639dd-0826-4e4f-aa37-7694a9ceb05b,None,2019-06-04 00:40:21.072065,None,673afaeb7cd432fdf7c060fca70d40364ce7646aedf6f4...,gs://stg-cdp-seattle.appspot.com/673afaeb7cd43...,...,2019-06-04 00:40:32.910534,2019-05-28 09:30:00,3966,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/CouncilBriefings...,http://video.seattle.gov:8080/media/council/br...,2019-06-04 00:40:31.798002,None,Council Briefing
0,0.935917,2019-06-04 00:49:28.535684,5970315e-cc33-4d84-8ee0-131274f2076d,c892939c-d84e-495c-a3ce-75526f3d7237,001909ae-f491-4fc4-8188-85aa4d524c83,None,2019-06-04 00:47:41.587430,None,15a707389674d13917b416d5cb51122f5824ec2450b90a...,gs://stg-cdp-seattle.appspot.com/15a707389674d...,...,2019-06-04 00:47:47.028966,2019-05-28 14:00:00,3964,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...,2019-06-04 00:03:50.029975,None,City Council
7,0.949865,2019-06-04 00:41:22.269522,7060eac6-0f96-44da-86c9-6f408335f451,1e465200-4c53-4d57-a158-282dfe21a692,cdb16bff-8aca-428c-a801-450f69884d5f,None,2019-06-04 00:40:43.635389,None,e8873216123755539a06535bd82e9a624e611c116ad8e7...,gs://stg-cdp-seattle.appspot.com/e887321612375...,...,2019-06-04 00:40:53.106354,2019-05-28 10:30:00,3965,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/selectcommitteem...,http://video.seattle.gov:8080/media/council/ho...,2019-06-04 00:40:51.977643,None,Select Committee on Citywide Mandatory Housing...
3,0.936662,2019-06-04 01:05:24.504101,83c21b25-47f4-4ef4-988e-49eae732e8d8,e3b56d4a-b66b-46dd-9a84-df2c2106a7c1,4f6d7c24-e7d0-4e28-a2b5-b3f97f19bda5,None,2019-06-04 01:04:47.738380,None,9d9fa6aa5aca27f76e3efe1685738f6e5df6d40a2ac674...,gs://stg-cdp-seattle.appspot.com/9d9fa6aa5aca2...,...,2019-06-04 01:05:12.913600,2019-05-28 09:30:00,3966,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/su...,2019-06-04 00:40:31.798002,None,Council Briefing


### A function to download the most recent transcripts

To make gathering the most recent transcripts easier, let's wrap this example up with a function to do all this for us.

**Note:** The function defined below is available in the `cdptools.utils` module.

`from cdptools.utils import download_most_recent_transcripts`

In [7]:
from pathlib import Path
from typing import Optional, Union

from cdptools.databases import Database
from cdptools.file_stores import FileStore


def download_most_recent_transcripts(db: Database, fs: FileStore, save_dir: Optional[Union[str, Path]] = None) -> Path:
    """
    Download the most recent versions of event transcripts.

    :param db: An already initialized `Database` object to query against.
    :param fs: An alreay initialized `FileStore` object to query against.
    :param save_dir: Path to a directory to save the transcripts and the dataset manifest.
    :return: Fully resolved path where transcripts and dataset manifest were stored.
    """
    # Use current directory is None provided
    if save_dir is None:
        save_dir = "."
    
    # Resolve save directory
    save_dir = Path(save_dir).resolve()
    
    # Make the save directory if not already exists
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Get transcript dataset
    transcripts = pd.DataFrame(db.select_rows_as_list("transcript"))
    events = pd.DataFrame(db.select_rows_as_list("event"))
    bodies = pd.DataFrame(db.select_rows_as_list("body"))
    files = pd.DataFrame(db.select_rows_as_list("file"))
    events = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
    transcripts = transcripts.merge(files, left_on="file_id", right_on="file_id", suffixes=("_transcript", "_file"))
    transcripts = transcripts.merge(events, left_on="event_id", right_on="event_id", suffixes=("_transcript", "_event"))
    
    # Group
    most_recent_transcripts = []
    grouped = transcripts.groupby("event_id")
    for name, group in grouped:
        most_recent = group.loc[group["created_transcript"].idxmax()]
        most_recent_transcripts.append(most_recent)
    
    most_recent = pd.DataFrame(most_recent_transcripts)
    
    # Begin storage
    most_recent.apply(
        lambda r: fs.download_file(r["filename"], save_dir / r["filename"], overwrite=True), 
        axis=1
    )
    
    # Write manifest
    most_recent.to_csv(save_dir / "transcript_manifest.csv", index=False)
    
    return save_dir

In [8]:
save_dir = download_most_recent_transcripts(db, fs, "most_recent_seattle_transcripts")
save_dir

PosixPath('/Users/maxfield/Desktop/active/cdp/cdptools/examples/most_recent_seattle_transcripts')